In [246]:
import pandas as pd
import requests as req
import bs4 as bs
import pydeck as pdk

In [269]:
url = 'https://www.bbc.com/russian/features-51979104'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = req.get(url, headers=headers)
soup = bs.BeautifulSoup(r.text, 'lxml')
region = soup.find_all('td', {'class': 'table__cell table__cell--region'})
infected = soup.find_all('td', {'class': 'table__cell table__cell--infected'})
recovered = soup.find_all('td', {'class': 'table__cell table__cell--infected'})
per100t = soup.find_all('td', {'class': 'table__cell table__cell--per100t'})
all_russia = 0
city = []
lons = []
lats = []
for i in range(len(region)):
    region[i] = region[i].get_text().strip().replace('\xa0', ' ').replace('АО', "автономный округ")
    infected[i] = int(infected[i].get_text().strip().replace('\xa0', ' ').replace(' ', ''))
    recovered[i] = int(recovered[i].get_text().strip().replace('\xa0', ' ').replace(' ', ''))
    per100t[i] = float(per100t[i].get_text().strip().replace('\xa0', '').replace(',', '.'))
    if region[i] == 'Еврейская автономный округ':
        region[i] = 'Еврейский автономный округ'
    elif region[i] == 'Москва':
        city.append('Москва')
        lats.append(55.5735)
        lons.append(37.5)
        continue
    elif region[i] == 'Санкт-Петербург':
        city.append('Санкт-Петербург')
        lats.append(59.921977)
        lons.append(30.3449437)
        continue
    elif region[i] == 'Вся Россия':
        all_russia = i
        city.append('')
        lats.append(0)
        lons.append(0)
        continue
    url = 'https://ru.wikipedia.org/wiki/{}'.format(region[i])
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = req.get(url, headers=headers)
    soupr = bs.BeautifulSoup(r.text, 'lxml')
    try:
        s = soupr.find_all('table', {'class': 'infobox'})[0]
        geo = str(s.find_all('span', {'class': 'geo-yandex'})[0].find_all('a', {'class': 'external text'})[0]['href'])
        geo = geo[geo.find('?ll=') + len('?ll='):geo.find('&')]
        lon, lat = map(float, geo.split(','))
        lats.append(lat)
        lons.append(lon)
    except:
        lats.append(0)
        lons.append(0)
    try:
        s = str(soupr.find_all('table', {'class': 'infobox'})[0])
    except:
        print(s)
    if s.find('Столица') == -1:
        if s.find('Административный центр') != -1:
            s = s[s.find('Административный центр'):]
        elif s.find('Адм.') != -1:
            s = s[s.find('Адм.'):]
        else:
            s = s[s.find('Областной центр'):]
    else:
        s = s[s.find('Столица'):]
    s = s[s.find('/th>') + len('/th>'):]
    s = bs.BeautifulSoup(s, 'lxml')
    try:
        city.append(s.find_all('span', {'class': 'no-wikidata'})[0].get_text().strip())
    except:
        city.append('')
    

In [271]:
url = 'https://aviateka.su/kody-aeroportov-iata-icao-rf/'
r = req.get(url, headers=headers)
soup = bs.BeautifulSoup(r.text, 'lxml')
c = 0
strs = soup.find_all('td')

from_city_to_iata = {}

for i in range(len(strs)):
    if strs[i].get_text().find('Аэропорт') != -1:
        strs[i] = str(strs[i])
        strs[i] = strs[i][strs[i].find('Аэропорт ') + len('Аэропорт '):]
        strs[i] = strs[i][:strs[i].find('<')]
        if strs[i] in city:
            c += 1
            from_city_to_iata[strs[i]] = strs[i + 1].get_text().strip()
iata = []

for i in city:
    if i in from_city_to_iata:
        iata.append(from_city_to_iata[i])
    else:
        iata.append('')
data = pd.DataFrame({'city': city, 'iata': iata, 'infected': infected, 'recovered': recovered, 'per100t': per100t, 'lat': lats, 'lon': lons})
data['iata'] = data.apply(lambda row: row['iata'] if row['city'] != 'Москва' else 'VKO', axis=1)
data['iata'] = data.apply(lambda row: row['iata'] if row['city'] != 'Екатеринбург' else 'SVX', axis=1)
data['iata'] = data.apply(lambda row: row['iata'] if row['city'] != 'Санкт-Петербург' else 'LED', axis=1)
data['iata'] = data.apply(lambda row: row['iata'] if row['city'] != 'Новосибирск' else 'OVB', axis=1)
data = data.loc[data.index!=all_russia]
data = data.loc[data.iata != '']
data = data.loc[data.city != '']
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data = data.sort_values(by=['city']).reset_index()
data = data.drop(columns=['index'])
data.to_csv('russia_cities.csv')
data

,city,iata,infected,recovered,per100t,lat,lon
0,Абакан,ABA,12063,12063,2258.2,53.500000,90.000000
1,Анадырь,DYR,389,389,766.9,66.252781,172.001000
2,Архангельск,ARH,31230,31230,2859.2,63.500000,43.000000
3,Астрахань,ASF,13715,13715,1363.4,47.233330,47.233330
4,Барнаул,BAX,26129,26129,1127.7,52.766670,82.616670
5,Белгород,EGO,16207,16207,1047.3,50.766670,37.450000
6,Благовещенск,BQS,11380,11380,1439.3,53.550000,127.833330
7,Брянск,BZK,17287,17287,1449.6,52.950000,33.400000
8,Владивосток,VVO,21916,21916,1156.3,45.333330,134.666670
9,Владикавказ,OGZ,9366,9366,1343.6,43.183330,44.233330
